In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [2]:
timestamp = '20240926-122824'

# fpath = os.path.join(DATA_DIR,'processed',f'formatted_state_fips_{timestamp}.csv')
# df_fips = pd.read_csv(fpath)

# fpath = os.path.join(DATA_DIR,'processed',f'formatted_COVIDhub-ensemble_{timestamp}.csv')
# df_hub = pd.read_csv(fpath)

# fpath = os.path.join(DATA_DIR,'processed',f'formatted_nytimes-us-counties_{timestamp}.csv')
# df_nyt = pd.read_csv(fpath)

fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_{timestamp}.csv')
df = pd.read_csv(fpath)

# display(df_fips.head())
# display(df_hub, df_nyt, df_acs)

In [3]:
dup_dict = {}

for name, ser in df.items():
#     print(series_name)
    for name2, ser2 in df.items():
        if name==name2:
            break
        else:
            if ser.equals(ser2):
                if name not in dup_dict.keys():
                    dup_dict[name] = [name2]
                else:
                    dup_dict[name].append(name2)
#     print(series)

In [4]:
df_dict = pd.DataFrame.from_dict(dup_dict, orient='index') 
fpath = "../constants_and_dicts/acs_duplicate_covs_dict.csv"
df_dict.to_csv(fpath,header=False)

In [5]:
import json

fpath = "../constants_and_dicts/acs_duplicate_covs_dict.json" 
with open(fpath, 'w') as fp:
    json.dump(dup_dict, fp)
with open(fpath, 'r') as json_file:
    data = json.load(json_file)
data

{'POP_x3': ['POP_x2'],
 'HH_x6': ['HH_x5'],
 'POP_16p_EMP_x8': ['POP_16p_EMP_x7'],
 'IND_CONS_x8': ['IND_CONS_x7'],
 'IND_MAN_x8': ['IND_MAN_x7'],
 'IND_WHT_x8': ['IND_WHT_x7'],
 'IND_RETT_x8': ['IND_RETT_x7'],
 'IND_INF_x8': ['IND_INF_x7'],
 'IND_OSER_x8': ['IND_OSER_x7'],
 'IND_PUBA_x8': ['IND_PUBA_x7'],
 'HU_OCC': ['HH_x5', 'HH_x6'],
 'POP_16p_EMP_x10': ['POP_16p_EMP_x7', 'POP_16p_EMP_x8'],
 'POP_16p_EMP_x11': ['POP_16p_EMP_x7', 'POP_16p_EMP_x8', 'POP_16p_EMP_x10'],
 'POP_16p_EMP': ['POP_16p_EMP_x7',
  'POP_16p_EMP_x8',
  'POP_16p_EMP_x10',
  'POP_16p_EMP_x11'],
 'OCC_SERV_HCS_x12': ['OCC_SERV_HCS_x11'],
 'OCC_SERV_FPS_x12': ['OCC_SERV_FPS_x11'],
 'OCC_SERV_BGM_x12': ['OCC_SERV_BGM_x11'],
 'OCC_SERV_PCS_x12': ['OCC_SERV_PCS_x11'],
 'OCC_SALES_SR_x12': ['OCC_SALES_SR_x11'],
 'OCC_SALES_OAS_x12': ['OCC_SALES_OAS_x11'],
 'OCC_NRCM_FFF_x12': ['OCC_NRCM_FFF_x11'],
 'OCC_NRCM_CE_x12': ['OCC_NRCM_CE_x11'],
 'OCC_NRCM_IMR_x12': ['OCC_NRCM_IMR_x11'],
 'OCC_PTMM_P_x12': ['OCC_PTMM_P_x11'],
 '

In [ ]:
dff = pd.read_csv(fpath, index_col=0)
d = dff.to_dict("split")
d = dict(zip(d["index"], d["data"]))
d


In [ ]:
dup_dict

In [ ]:
dup_dict = {}

for col in df_acs.columns:
    ser = df_acs
    print(col)

In [ ]:
[x for x in df_acs.columns if 'POP' in x]

In [ ]:
df_acs
# Transpose the DataFrame to make columns as rows
dff = df_acs.transpose()

# Find duplicate columns (excluding the first occurrence)
duplicate_columns = dff.duplicated(keep='first')

# Get the unique column names
unique_columns = dff[~duplicate_columns].index

# Transpose the DataFrame back and keep only the unique columns
df_output = df_acs[unique_columns].copy()

# Print the resulting DataFrame
print(df_output)

In [ ]:
df_acs.columns

In [ ]:
## First, check for overlap of Fips codes between df_hub and df_nyt ...

fipss_hub = df_hub.Fips.nunique() 
fipss_nyt = df_nyt.Fips.nunique()

In [ ]:
fipss_hub, fipss_nyt

In [ ]:
import json

with open('..raw_data/county_covariates.json') as f:
    d = json.load(f)

county_covs = []
for entry in d:
    county_covs.append(entry)

df_pop = pd.DataFrame(columns=['Postal','Fips','Pop'])

for idx in range(len(county_covs)):
    pop = county_covs[idx]['population']['2019']
    fips = county_covs[idx]['fips']
    state = county_covs[idx]['state']
    df_pop.loc[idx] = pd.Series({'Postal':state, 'Fips':str(fips), 'Pop':pop})
    
    
state_fips = pd.read_csv('constants/state_fips.csv')

df_pop = df_pop.merge(state_fips, on='Postal')
df_pop.drop('FIPS',inplace=True,axis=1)
df_pop